In [4]:
import json

from pyspark.sql import Row

### W205: Project 2

As directed, we use the following command to download the data:
```
    curl -L -o assessment-attempts-20180128-121051-nested.json https://goo.gl/ME6hjp`
```

### 1a) Command to bring up the cluster:
```
    docker-compose up -d
```
### 1b) Optionally, check that the cluster is up by running the following 2 commands:
```
    docker-compose ps
    docker-compose ps -a
```

### 2a) command to create the kafka topic with a the name "assessments":
```
docker-compose exec kafka kafka-topics --create --topic assessments --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181
```

```
```

### 2b) We can describe the topic using the command,
```
docker-compose exec kafka kafka-topics --describe --topic assessments --zookeeper zookeeper:32181
```

### 3) command to publish the assessments json data to the kafka topic using kafkacat
```
docker-compose exec mids bash -c "cat /w205/project-2-asozer/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessments"
```

### 4) command to shutdown the cluster:
```
docker-compose down
```

```
```

### 4b) We can check that the cluster is teared down using:

```
docker-compose ps
```
and 

```
docker ps -a 
```

### 1) create a data frame by subscribing to the kafka topic

In [12]:
raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 

In [13]:
raw_assessments.show(20)

+----+--------------------+-----------+---------+------+--------------------+-------------+
| key|               value|      topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----------+---------+------+--------------------+-------------+
|null|[7B 22 6B 65 65 6...|assessments|        0|     0|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     1|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     2|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     3|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     4|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     5|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     6|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     7|1969-12-31 23:59:...|   

### 2) convert the json data as a string to a new dataframe

In [15]:
raw_assessments.cache()

assessments = raw_assessments.select(raw_assessments.value.cast('string'))
assessments.show(20)

+--------------------+
|               value|
+--------------------+
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
+--------------------+
only showing top 20 rows



### 3) extract / unroll the json data into new dataframes to answer you business questions

In [16]:
extracted_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()

extracted_assessments.registerTempTable('assessments')
extracted_assessments.show()

+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|        base_exam_id|certification|           exam_name|   keen_created_at|             keen_id|    keen_timestamp|max_attempts|           sequences|          started_at|        user_exam_id|
+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717442.735266|5a6745820eb8ab000...| 1516717442.735266|         1.0|Map(questions -> ...|2018-01-23T14:23:...|6d4089e4-bde5-4a2...|
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717377.639827|5a674541ab6b0a000...| 1516717377.639827|         1.0|Map(questions -> ...|2018-01-23T14:21:...|2fec1534-b41f-441...|
|4beeac16-bb83-4d5...|        false

### 4 and 5) Register Dataframes as Temporary Tables to Allow in Memory Queries Against them AND Perform SQL Queries Against the Dataframes You Registered

In [10]:
extracted_assessments.registerTempTable('assessments')

In [11]:
assessments

DataFrame[value: string]

In [28]:
raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 

raw_assessments.cache()

assessments = raw_assessments.select(raw_assessments.value.cast('string'))

import json

from pyspark.sql import Row

extracted_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()

extracted_assessments.registerTempTable('assessments')

spark.sql("select keen_id from assessments limit 10").show()

spark.sql("select keen_timestamp, sequences.questions[0].user_incomplete from assessments limit 10").show()

+--------------------+
|             keen_id|
+--------------------+
|5a6745820eb8ab000...|
|5a674541ab6b0a000...|
|5a67999d3ed3e3000...|
|5a6799694fc7c7000...|
|5a6791e824fccd000...|
|5a67a0b6852c2a000...|
|5a67b627cc80e6000...|
|5a67ac8cb0a5f4000...|
|5a67a9ba060087000...|
|5a67ac54411aed000...|
+--------------------+

+------------------+-------------------------------------------------------+
|    keen_timestamp|sequences[questions] AS `questions`[0][user_incomplete]|
+------------------+-------------------------------------------------------+
| 1516717442.735266|                                                   true|
| 1516717377.639827|                                                  false|
| 1516738973.653394|                                                  false|
|1516738921.1137421|                                                  false|
| 1516737000.212122|                                                  false|
| 1516740790.309757|                                         

### What are the exam_names?

In [40]:
spark.sql("select exam_name from assessments").show()


+--------------------+
|           exam_name|
+--------------------+
|Normal Forms and ...|
|Normal Forms and ...|
|The Principles of...|
|The Principles of...|
|Introduction to B...|
|        Learning Git|
|Git Fundamentals ...|
|Introduction to P...|
|Intermediate Pyth...|
|Introduction to P...|
|A Practical Intro...|
|Git Fundamentals ...|
|Introduction to M...|
|   Python Epiphanies|
|Introduction to P...|
|Python Data Struc...|
|Python Data Struc...|
|Working with Algo...|
|Learning iPython ...|
|   Python Epiphanies|
+--------------------+
only showing top 20 rows



### What are the highest number of students who took each exam?

In [44]:
spark.sql("select exam_name, count(*) as number_of_students from assessments group by exam_name order by number_of_students desc").show(truncate = False)


+--------------------------------------------------------------+------------------+
|exam_name                                                     |number_of_students|
+--------------------------------------------------------------+------------------+
|Learning Git                                                  |394               |
|Introduction to Python                                        |162               |
|Introduction to Java 8                                        |158               |
|Intermediate Python Programming                               |158               |
|Learning to Program with R                                    |128               |
|Introduction to Machine Learning                              |119               |
|Software Architecture Fundamentals Understanding the Basics   |109               |
|Beginning C# Programming                                      |95                |
|Learning Eclipse                                              |85          

### What are the lowest number of students who took each exam?

In [45]:
spark.sql("select exam_name, count(*) as number_of_students from assessments group by exam_name order by number_of_students asc").show(truncate = False)


+--------------------------------------------------+------------------+
|exam_name                                         |number_of_students|
+--------------------------------------------------+------------------+
|Learning to Visualize Data with D3.js             |1                 |
|Native Web Apps for Android                       |1                 |
|Nulls, Three-valued Logic and Missing Information |1                 |
|Operating Red Hat Enterprise Linux Servers        |1                 |
|Learning Spring Programming                       |2                 |
|Client-Side Data Storage for Web Developers       |2                 |
|Understanding the Grails 3 Domain Model           |2                 |
|The Closed World Assumption                       |2                 |
|Hibernate and JPA Fundamentals                    |2                 |
|What's New in JavaScript                          |2                 |
|Arduino Prototyping Techniques                    |2           

In [46]:
def my_lambda_sequences_id(x):
    raw_dict = json.loads(x.value)
    my_dict = {"keen_id" : raw_dict["keen_id"], "sequences_id" : raw_dict["sequences"]["id"]}
    return Row(**my_dict)

assessments.show()

+--------------------+
|               value|
+--------------------+
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
+--------------------+
only showing top 20 rows



In [34]:
my_sequences = assessments.rdd.map(my_lambda_sequences_id).toDF()



In [35]:
my_sequences.registerTempTable('sequences')


In [36]:
spark.sql("select sequences_id from sequences limit 10").show()

spark.sql("select a.keen_id, a.keen_timestamp, s.sequences_id from assessments a join sequences s on a.keen_id = s.keen_id limit 10").show()

+--------------------+
|        sequences_id|
+--------------------+
|5b28a462-7a3b-42e...|
|5b28a462-7a3b-42e...|
|b370a3aa-bf9e-4c1...|
|b370a3aa-bf9e-4c1...|
|04a192c1-4f5c-4ac...|
|e7110aed-0d08-4cb...|
|5251db24-2a6e-424...|
|066b5326-e547-4da...|
|8ac691f8-8c1a-403...|
|066b5326-e547-4da...|
+--------------------+

+--------------------+------------------+--------------------+
|             keen_id|    keen_timestamp|        sequences_id|
+--------------------+------------------+--------------------+
|5a17a67efa1257000...|1511499390.3836269|8ac691f8-8c1a-403...|
|5a26ee9cbf5ce1000...|1512500892.4166169|9bd87823-4508-4e0...|
|5a29dcac74b662000...|1512692908.8423469|e7110aed-0d08-4cb...|
|5a2fdab0eabeda000...|1513085616.2275269|cd800e92-afc3-447...|
|5a30105020e9d4000...|1513099344.8624721|8ac691f8-8c1a-403...|
|5a3a6fc3f0a100000...| 1513779139.354213|e7110aed-0d08-4cb...|
|5a4e17fe08a892000...|1515067390.1336551|9abd5b51-6bd8-11e...|
|5a4f3c69cc6444000...| 1515142249.858722|083844

In [37]:
def my_lambda_questions(x):
    raw_dict = json.loads(x.value)
    my_list = []
    my_count = 0
    for l in raw_dict["sequences"]["questions"]:
        my_count += 1
        my_dict = {"keen_id" : raw_dict["keen_id"], "my_count" : my_count, "id" : l["id"]}
        my_list.append(Row(**my_dict))
    return my_list

my_questions = assessments.rdd.flatMap(my_lambda_questions).toDF()

my_questions.registerTempTable('questions')

spark.sql("select id, my_count from questions limit 10").show()

spark.sql("select q.keen_id, a.keen_timestamp, q.id from assessments a join questions q on a.keen_id = q.keen_id limit 10").show()

+--------------------+--------+
|                  id|my_count|
+--------------------+--------+
|7a2ed6d3-f492-49b...|       1|
|bbed4358-999d-446...|       2|
|e6ad8644-96b1-461...|       3|
|95194331-ac43-454...|       4|
|95194331-ac43-454...|       1|
|bbed4358-999d-446...|       2|
|e6ad8644-96b1-461...|       3|
|7a2ed6d3-f492-49b...|       4|
|b9ff2e88-cf9d-4bd...|       1|
|bec23e7b-4870-49f...|       2|
+--------------------+--------+

+--------------------+------------------+--------------------+
|             keen_id|    keen_timestamp|                  id|
+--------------------+------------------+--------------------+
|5a17a67efa1257000...|1511499390.3836269|803fc93f-7eb2-412...|
|5a17a67efa1257000...|1511499390.3836269|f3cb88cc-5b79-41b...|
|5a17a67efa1257000...|1511499390.3836269|32fe7d8d-6d89-4db...|
|5a17a67efa1257000...|1511499390.3836269|5c34cf19-8cfd-4f5...|
|5a26ee9cbf5ce1000...|1512500892.4166169|0603e6f4-c3f9-4c2...|
|5a26ee9cbf5ce1000...|1512500892.4166169|26a06b

### Which keen.id has most and least questions?

keen.id's with the most questions:

In [48]:
spark.sql("select id, my_count from questions order by my_count").show()

+--------------------+--------+
|                  id|my_count|
+--------------------+--------+
|7bdbbf4a-b5d8-4c7...|       1|
|dccd58b0-251b-40f...|       1|
|d2ac7f0d-82bd-415...|       1|
|95194331-ac43-454...|       1|
|59d444b5-49fd-487...|       1|
|1f7c5def-904b-483...|       1|
|e272a3d1-bd67-4d2...|       1|
|fb07b16e-84a2-465...|       1|
|dee14932-a24e-4aa...|       1|
|fc3bdc54-04a8-4b4...|       1|
|861c3405-83fc-42f...|       1|
|fc3bdc54-04a8-4b4...|       1|
|f289b342-2871-4ab...|       1|
|247b4589-7f8c-4a4...|       1|
|aa7a63cd-7aa4-47f...|       1|
|26ddad33-aa1d-49e...|       1|
|e272a3d1-bd67-4d2...|       1|
|917b9413-f83b-4f2...|       1|
|c3e6e626-884a-4be...|       1|
|34eacafb-dd6a-424...|       1|
+--------------------+--------+
only showing top 20 rows



keen.id's with the least questions:

In [49]:
spark.sql("select id, my_count from questions order by my_count desc").show()

+--------------------+--------+
|                  id|my_count|
+--------------------+--------+
|8d5372d4-a63b-40c...|      20|
|7d527603-ed07-4d1...|      19|
|bdc4d043-b161-426...|      18|
|2494e12b-070e-458...|      17|
|a994e9ca-208a-40a...|      16|
|ebc3d26e-ed70-4cd...|      15|
|ddac0ade-2320-48d...|      14|
|0bc8696a-b9b0-43b...|      13|
|c27494a4-fe24-4a1...|      12|
|be969a50-0474-409...|      11|
|7193e678-1989-4bf...|      10|
|283dee13-3e46-480...|      10|
|e3cc7e9c-603a-48c...|      10|
|bc7ae396-a747-4f8...|      10|
|48f0cecb-80f7-4b4...|      10|
|7193e678-1989-4bf...|      10|
|7193e678-1989-4bf...|      10|
|283dee13-3e46-480...|      10|
|48f0cecb-80f7-4b4...|      10|
|bc7ae396-a747-4f8...|      10|
+--------------------+--------+
only showing top 20 rows



In [38]:
def my_lambda_correct_total(x):
    
    raw_dict = json.loads(x.value)
    my_list = []
    
    if "sequences" in raw_dict:
        
        if "counts" in raw_dict["sequences"]:
            
            if "correct" in raw_dict["sequences"]["counts"] and "total" in raw_dict["sequences"]["counts"]:
                    
                my_dict = {"correct": raw_dict["sequences"]["counts"]["correct"], 
                           "total": raw_dict["sequences"]["counts"]["total"]}
                my_list.append(Row(**my_dict))
    
    return my_list

my_correct_total = assessments.rdd.flatMap(my_lambda_correct_total).toDF()

my_correct_total.registerTempTable('ct')

spark.sql("select * from ct limit 10").show()

spark.sql("select correct / total as score from ct limit 10").show()

spark.sql("select avg(correct / total)*100 as avg_score from ct limit 10").show()

spark.sql("select stddev(correct / total) as standard_deviation from ct limit 10").show()

+-------+-----+
|correct|total|
+-------+-----+
|      2|    4|
|      1|    4|
|      3|    4|
|      2|    4|
|      3|    4|
|      5|    5|
|      1|    1|
|      5|    5|
|      4|    4|
|      0|    5|
+-------+-----+

+-----+
|score|
+-----+
|  0.5|
| 0.25|
| 0.75|
|  0.5|
| 0.75|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  0.0|
+-----+

+-----------------+
|        avg_score|
+-----------------+
|62.65699745547047|
+-----------------+

+-------------------+
| standard_deviation|
+-------------------+
|0.31086692286170553|
+-------------------+



### 6 perform a write to HDFS in parquet format for each data frame you created

In [50]:
raw_assessments.write.mode('overwrite').parquet("/tmp/raw_assessments")

In [ ]:
my_sequences.write.mode('overwrite').parquet("/tmp/sequences")

In [ ]:
my_questions.write.mode('overwrite').parquet("/tmp/questions")

In [ ]:
my_correct_total.write.mode('overwrite').parquet("/tmp/correct_total")